# Experiment 3
- Load the saved model and replace the output layer of the model, as well as the first two convolutional layers (keep the weights of all other layers).
- Train and evaluate the model on the cats and dogs dataset.

## Steps
Load the saved model from a file.

Freeze all layers except the first two convolutional layers and the output layer.

Replace the first two convolutional layers and the output layer.

Compile the model with a suitable loss function and optimizer.

Train and evaluate the model on the Cats vs. Dogs dataset.

### Load the saved model

In [2]:
import tensorflow as tf
from tensorflow import data as tf_data
from tensorflow import keras
from keras import layers

# Load the pre-trained model
saved_model_path = "models/experiment1_model.keras"
model = keras.models.load_model(saved_model_path)

### Modify the model

In [3]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # Change output layer for multi-class classification (120 dog breeds)
    outputs = layers.Dense(units, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [ ]:
# Create a fresh model with the same architecture
fresh_model = make_model(input_shape=(180, 180) + (3,), num_classes=1)
base_model = keras.Model(inputs=model.input, outputs=model.layers[-2].output)
base_model.trainable = False 

# Reset weights for layers 6 to 22
for i in range(6, 22):
    base_model.layers[i].set_weights(fresh_model.layers[i].get_weights())
    base_model.layers[i].trainable = True  # Make sure the layers are trainable

In [5]:
# Add new output layer (Binary Classification)
new_output = layers.Dense(1, activation="sigmoid")(base_model.output)

# Create a new model
new_model = keras.Model(inputs=base_model.input, outputs=new_output)

### Compile the model

In [6]:
new_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Modification for experiment 1
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)

### Create the dataset

In [ ]:
# Create the dataset
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x), y))

# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

Found 23422 files belonging to 2 classes.
Using 18738 files for training.
Using 4684 files for validation.


: 

### Train and evaluate the model

In [ ]:
epochs = 50
callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="models/experiment3_epoch_{epoch}.keras")
]


new_model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=callbacks
)

new_model.save("models/experiment3_model.keras")


Epoch 1/50


c:\Users\Utilisateur\Documents\GitHub\DL_Assignements\.venv\Lib\site-packages\keras\src\backend\tensorflow\nn.py:780: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


147/147 ━━━━━━━━━━━━━━━━━━━━ 857s 6s/step - acc: 0.6036 - loss: 0.6986 - val_acc: 0.5056 - val_loss: 0.7306
Epoch 2/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 824s 6s/step - acc: 0.6785 - loss: 0.5930 - val_acc: 0.5175 - val_loss: 0.7000
Epoch 3/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 824s 6s/step - acc: 0.7159 - loss: 0.5538 - val_acc: 0.5009 - val_loss: 0.6997
Epoch 4/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 824s 6s/step - acc: 0.7322 - loss: 0.5288 - val_acc: 0.5026 - val_loss: 0.7279
Epoch 5/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 824s 6s/step - acc: 0.7489 - loss: 0.5038 - val_acc: 0.6806 - val_loss: 0.5672
Epoch 6/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 824s 6s/step - acc: 0.7691 - loss: 0.4765 - val_acc: 0.7686 - val_loss: 0.4802
Epoch 7/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 824s 6s/step - acc: 0.7677 - loss: 0.4736 - val_acc: 0.8096 - val_loss: 0.4352
Epoch 8/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 824s 6s/step - acc: 0.7926 - loss: 0.4444 - val_acc: 0.8064 - val_loss: 0.4211
Epoch 9/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 825s 6s/step - acc: